In [70]:
require('ranger')
require('party')

In [71]:
X.train.dgi <- read.csv("../intermediate/02_enhancer/permuted1_X_train.csv", header = FALSE)
y.train.dgi <- read.csv("../intermediate/02_enhancer/permuted1_y_train.csv", header = FALSE)
noisy_features <- read.csv('../intermediate/02_enhancer/permuted1_noisy_features.csv', header = FALSE)

set.seed(2356789)
data.dgi <- data.frame(X.train.dgi, y = y.train.dgi[, 1])
dgi.rf = ranger(y ~ ., data = data.dgi, 
       num.trees = 100,
       mtry = 3,
       importance = "impurity",
       max.depth = 20)
imp_org = dgi.rf$variable.importance
imp_org = imp_org/sum(imp_org)

dgi.perm = ranger(y ~ ., data = data.dgi, 
                num.trees = 100,
                mtry = 3,
                importance = "impurity_corrected",
                 min.node.size = 20)
imp_ranger = dgi.perm$variable.importance
imp_ranger = imp_ranger/sum(imp_ranger)

dgi.cforest = cforest(y~., data = data.dgi,
                           control = cforest_control(ntree = 100, 
                                                     mtry = 3, 
                                                     maxdepth = 20))
imp_cforest = varimp(dgi.cforest)
imp_cforest = imp_cforest/sum(imp_cforest)

imp = data.frame(imp_org, imp_ranger, imp_cforest)
write.csv(imp, "imp.csv")

ERROR: Error in ranger(y ~ ., data = data.dgi, num.trees = 100, mtry = 3, importance = "impurity", : unused argument (max.depth = 20)


In [72]:
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 16.04.6 LTS

Matrix products: default
BLAS: /home/ubuntu/anaconda3/envs/R/lib/R/lib/libRblas.so
LAPACK: /home/ubuntu/anaconda3/envs/R/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats4    grid      stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] pROC_1.12.1          party_1.3-0          strucchange_1.5-1   
 [4] sandwich_2.4-0       zoo_1.8-3            modeltools_0.2-22   
 [7] mvtnorm_1.0-8        ranger_0.10.1        RevoUtils_11.0.1    
[10] RevoUtilsMath_11.0.0

loaded vi

In [24]:
cor(imp, 1 - noisy_features)

,V1
imp_org,-0.1397322
imp_ranger,0.2618766
imp_cforest,0.1636625


In [36]:
library(pROC)
roc_obj <- roc(array(noisy_features), array(noisy_features))
auc(roc_obj)

ERROR: Error in roc.default(array(noisy_features), array(noisy_features)): 'response' must have two levels


In [65]:
pos.scores <- imp$imp_org[noisy_features == 0]
neg.scores <- imp$imp_org[noisy_features == 1]
mean(sample(pos.scores,100000,replace=T) > sample(neg.scores,100000,replace=T))

[1] 0.32073

In [69]:
dim(X.train.dgi)

[1] 3912   80